In [1]:
class UnstructuredDb():
    def __init__(self,path:str="chatbot/",table:str='Psychological_counseling'):
        import chromadb
        import os
        from chromadb.db.base import UniqueConstraintError
        from chromadb.utils import embedding_functions
        
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            print(f"Directory '{path}' created successfully.")

         
        
        client = chromadb.PersistentClient(path=path)
        em = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="Huffon/sentence-klue-roberta-base")
        try:
            self.collection = client.create_collection(name=table, embedding_function=em)
        except UniqueConstraintError: 
            self.collection = client.get_collection(name=table, embedding_function=em)
            
    def get(self,question:str)->[str]:
        
        results=self.collection.query(query_texts=[question],n_results=1)
        return results


    def add(self,key:str=None,question:str=None,answer:str=None):

        self.collection.add(
        documents=[question],
        metadatas=[answer],
        ids=[key]
        )

In [2]:
db=UnstructuredDb()

No sentence-transformers model found with name Huffon/sentence-klue-roberta-base. Creating a new one with MEAN pooling.


In [11]:
from flask import Flask, request, jsonify, render_template


app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return render_template("chatbot.html")


@app.route('/question', methods=['POST'])
def answer():
    # 클라이언트로부터 받은 데이터를 JSON 형태로 파싱
    data = request.get_json()

    result=db.get(data.get('text'))
    result=list(result['metadatas'][0][0].keys())[0]
    return jsonify(response=result)

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/May/2024 04:01:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:01:35] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:01:39] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:01:50] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:03] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:15] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:22] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:26] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:39] "POST /question HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2024 04:02:46] "POST /question HTTP/1.1" 200 -


In [ ]:
# import pandas as pd
# import shutil

# data=pd.read_csv('wellness_dataset.csv')
# for index, row in data.iterrows():
#     print((index/1034)*100)
#     db.add(answer={row['챗봇']:row['구분']},question=row['유저'],key=str(index))

# # 디렉토리를 ZIP 파일로 압축
# shutil.make_archive('chatbot', 'zip', './chatbot')
